In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import random
from sklearn.pipeline import Pipeline
from pathlib import Path


from os.path import join, isfile
from os import path, scandir, listdir


import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.layers import Input, Dense, BatchNormalization, Dropout, Embedding,  Flatten
from tensorflow.keras.models import Model, Sequential
from keras.callbacks import ReduceLROnPlateau
from keras.optimizers import RMSprop
import keras_tuner as kt

from tensorflow.data import Dataset
from sklearn.preprocessing import QuantileTransformer,  KBinsDiscretizer
from sklearn.model_selection import StratifiedKFold
from tensorflow import keras
from sklearn import metrics
from sklearn.impute import SimpleImputer

from sklearn.model_selection import train_test_split

from sklearn.metrics import roc_auc_score

import gc
import warnings
warnings.simplefilter('ignore')

# Parameters

In [2]:
target = 'target'

DEBUG = False

if DEBUG:
    N_ESTIMATORS = 1
    N_SPLITS = 2
    SEED = 2017
    CVSEED = 2017
    EARLY_STOPPING_ROUNDS = 1
    VERBOSE = 100
    BINS = 128
    #N_ITERS = 2
else:
    N_SPLITS = 5
    N_ESTIMATORS = 20000
    EARLY_STOPPING_ROUNDS = 300
    VERBOSE = 1000
    SEED = 2017
    CVSEED = 2017
    BINS = 128
    #N_ITERS = 10

In [3]:
def set_seed(seed=2017):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    tf.compat.v1.set_random_seed(seed)
    
set_seed(SEED)

# Load Dataset

In [4]:
INPUT = Path("../input/tabular-playground-series-oct-2021")

train = pd.DataFrame(pd.read_csv(INPUT / "train.csv")[target])
#test = pd.read_csv(INPUT / "test.csv")
test = pd.DataFrame()
sub = pd.read_csv(INPUT / "sample_submission.csv")

In [5]:
def list_all_files(location='../input/tps-oct-lv0', pattern=None, recursive=True):
    """
    This function returns a list of files at a given location (including subfolders)
    
    - location: path to the directory to be searched
    - pattern: part of the file name to be searched (ex. pattern='.csv' would return all the csv files)
    - recursive: boolean, if True the function calls itself for every subdirectory it finds
    """
    subdirectories= [f.path for f in scandir(location) if f.is_dir()]
    files = [join(location, f) for f in listdir(location) if isfile(join(location, f))]
    if recursive:
        for directory in subdirectories:
            files.extend(list_all_files(directory))
    if pattern:
        files = [f for f in files if pattern in f]
    return files

In [6]:
names = ['bizen', 'henke', 'hamza', '28smiles','kashif', 'kosta', 'kaveh', 'dlaststark', 'pca', 'xgb2']

In [7]:
namesec = ['lonnie', 'hgb', 'xgb_d2s', 'ctb_d2s']

In [8]:
pred = list_all_files(pattern='oof')


for i in range(len(names)):
    avv = []
    
    for file in pred:
        if names[i] in file.split('/')[3]:
            avv.append(np.load(file))
    train[names[i]] = np.mean(avv, axis=0)
            
pred = list_all_files(location='../input/tps-oct-lv0-sec', pattern='oof')

for i in range(len(namesec)):
    avv = []
    
    for file in pred:
        if namesec[i] in file.split('/')[3]:
            avv.append(np.load(file))
    train[namesec[i]] = np.mean(avv, axis=0)
            
            
    
train.columns

Index(['target', 'bizen', 'henke', 'hamza', '28smiles', 'kashif', 'kosta',
       'kaveh', 'dlaststark', 'pca', 'xgb2', 'lonnie', 'hgb', 'xgb_d2s',
       'ctb_d2s'],
      dtype='object')

In [9]:
pred = list_all_files(pattern='pred')


for i in range(len(names)):
    avv = []
    
    for file in pred:
        if names[i] in file.split('/')[3]:
            avv.append(np.load(file))
    test[names[i]] = np.mean(avv, axis=0)
            
pred = list_all_files(location='../input/tps-oct-lv0-sec', pattern='pred')

for i in range(len(namesec)):
    avv = []
    
    for file in pred:
        if namesec[i] in file.split('/')[3]:
            avv.append(np.load(file))
    test[namesec[i]] = np.mean(avv, axis=0)
    
test.columns

Index(['bizen', 'henke', 'hamza', '28smiles', 'kashif', 'kosta', 'kaveh',
       'dlaststark', 'pca', 'xgb2', 'lonnie', 'hgb', 'xgb_d2s', 'ctb_d2s'],
      dtype='object')

In [10]:
avv = []
for i in range(5):
    avv.append(np.load("../input/tps-oct-lv0/"+str(i+2017)+"lgb_oof.npy"))
train['lgb'] = np.mean(avv, axis=0)

avv = []
for i in range(5):
    avv.append(np.load("../input/tps-oct-lv0/agg"+str(i+1)+"_xgb_oof.npy"))
train['xgb'] = np.mean(avv, axis=0)

avv = []
for i in range(5):
    avv.append(np.load("../input/tps-oct-lv0/"+str(i+2017)+"lgb_pred.npy"))
test['lgb'] = np.mean(avv, axis=0)

avv = []
for i in range(5):
    avv.append(np.load("../input/tps-oct-lv0/agg"+str(i+1)+"_xgb_pred.npy"))
test['xgb'] = np.mean(avv, axis=0)

# Preprocessing

In [11]:
features = train.columns[1:]

pipe = Pipeline([
       # ('imputer', SimpleImputer(strategy='median',missing_values=np.nan)),
        ("scaler", QuantileTransformer(n_quantiles=BINS,output_distribution='normal')),
        ('bin', KBinsDiscretizer(n_bins=BINS, encode='ordinal',strategy='uniform'))
        ])
#train[features] = pipe.fit_transform(train[features])
#test[features] = pipe.transform(test[features])

In [12]:
test[features]

,bizen,henke,hamza,28smiles,kashif,kosta,kaveh,dlaststark,pca,xgb2,lonnie,hgb,xgb_d2s,ctb_d2s,lgb,xgb
0,0.728526,0.722509,0.752387,0.705870,0.727055,0.735858,0.740599,0.745366,0.736168,0.731839,0.749371,0.752841,0.741152,0.750761,0.737219,0.748278
1,0.243854,0.245698,0.242451,0.235202,0.233988,0.240009,0.249772,0.248024,0.250602,0.235529,0.249697,0.260499,0.234527,0.231629,0.230626,0.233039
2,0.913063,0.912550,0.909911,0.919106,0.906819,0.910741,0.907417,0.903758,0.906905,0.906797,0.909948,0.897610,0.912578,0.898247,0.904611,0.901715
3,0.835983,0.832255,0.853693,0.817317,0.834168,0.835611,0.856448,0.843686,0.855629,0.838438,0.854144,0.850890,0.847592,0.856051,0.848851,0.853474
4,0.269920,0.276108,0.249503,0.270417,0.256404,0.265450,0.266382,0.268529,0.263090,0.260674,0.283369,0.263720,0.256101,0.277291,0.255942,0.262485
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
499995,0.975451,0.975318,0.968899,0.976103,0.966070,0.970023,0.970539,0.963713,0.970643,0.968542,0.965328,0.970665,0.969341,0.957678,0.965190,0.966362
499996,0.839360,0.841742,0.827404,0.850740,0.839601,0.834818,0.830072,0.828243,0.830860,0.830597,0.832374,0.810808,0.841105,0.825190,0.830506,0.830808
499997,0.293532,0.286726,0.299620,0.300556,0.289637,0.286823,0.284007,0.290569,0.284187,0.299252,0.287720,0.311128,0.280542,0.275137,0.288725,0.289752
499998,0.513656,0.505806,0.499243,0.526598,0.517690,0.512041,0.488363,0.496002,0.487719,0.531208,0.512495,0.495243,0.490784,0.506517,0.506413,0.505003


In [13]:
train[target]

0         1
1         1
2         1
3         1
4         1
         ..
999995    1
999996    0
999997    0
999998    1
999999    0
Name: target, Length: 1000000, dtype: int64

# Model

In [14]:
def make_model():
    
    lr = 0.0157
    dropout =0.125
    embed_dim =4
    hidden_dim = 144
    n_layers = 3
    act = 'swish'
    #dstep = hp.Int('decay_steps', min_value=2000, max_value=4000, step=200)
    drate = 0.885
    eps =3.6e-08
        
    
    inputs = Input(train[features].shape[1:])
    X = Embedding(input_dim=BINS, output_dim=embed_dim, embeddings_initializer = "glorot_normal")(inputs)
    X = Dropout(dropout)(X)
    #X = BatchNormalization()(X)
    X = Flatten()(X)
    
    for i in range(n_layers):
        #units = hp.Int('units_{i}'.format(i=i), min_value=8, max_value=256, step=8)
        X = layers.Dense(hidden_dim/(2**i), activation=act, kernel_initializer=tf.keras.initializers.GlorotNormal())(X)
        X = Dropout(dropout)(X)
        #X = BatchNormalization()(X)
    outputs = layers.Dense(1, activation='sigmoid', kernel_initializer=tf.keras.initializers.GlorotNormal())(X)
    model = keras.Model(inputs, outputs)
    
    #learning_rate = hp.Float('learning_rate', min_value=3e-4, max_value=3e-3)
    lr_schedule = keras.optimizers.schedules.ExponentialDecay(
        initial_learning_rate=lr,
        decay_steps = 400,
        decay_rate= drate)

    optimizer = keras.optimizers.Adam(learning_rate=lr_schedule, epsilon=eps)
    model.compile(loss=keras.losses.binary_crossentropy,
                  optimizer=optimizer,
                  metrics=[tf.keras.metrics.AUC(name='aucroc')])
    #model.summary()
    return model

# NN

In [15]:
def prediction(x, y, batch_size=1024, epochs=100):
    cv = StratifiedKFold(n_splits=N_SPLITS, shuffle=True, random_state=CVSEED)
    val_losses = np.zeros(x.shape[0])
    nn_pred = np.zeros(test.shape[0])
    
    for fold, (train_indices, val_indices) in enumerate(cv.split(x, y)):
      print(f"===== fold {fold} =====")
      x_train, x_valid = x.iloc[train_indices], x.iloc[val_indices]
      y_train, y_valid = y.iloc[train_indices], y.iloc[val_indices]
      x_test = test[features]
    
      gc.collect()
        
      x_train[features] = pipe.fit_transform(x_train[features])
      x_valid[features] = pipe.transform(x_valid[features])
      x_test[features] = pipe.transform(x_test[features])
        
      
    
      model = make_model()
      model.fit( x_train[features], y_train, 
                validation_data=(x_valid[features] , y_valid),
                shuffle=True,
                verbose=0,
                #callbacks=[model_checkpoint_callback],
                callbacks=[
                #tf.keras.callbacks.ReduceLROnPlateau(monitor='val_aucroc', mode='max', patience=2),
                tf.keras.callbacks.EarlyStopping(monitor='val_aucroc', mode='max', patience=5)  ],
                batch_size=batch_size, 
                epochs=epochs)
      val_losses[val_indices] += model.predict(x_valid[features] )[:,-1]
      nn_pred += model.predict(x_test[features] )[:,-1]

      auc = roc_auc_score(y_valid, val_losses[val_indices])
      print(f"fold {fold} - nn auc: {auc:.6f}")   
    
      del model
      gc.collect()
     
    nn_pred /= N_SPLITS
    print(f"oof nn_auc = {roc_auc_score(y, val_losses)}")
    
    np.save("nn_oof.npy", val_losses)
    np.save("nn_pred.npy", nn_pred)

    gc.collect()
    return nn_pred

In [16]:
nn_pred = prediction(x=train[features], y=train[target], 
             batch_size=1024, 
             epochs=100,
             #validation_data=(x_val, y_val),
            )

===== fold 0 =====
fold 0 - nn auc: 0.856657
===== fold 1 =====
fold 1 - nn auc: 0.857934
===== fold 2 =====
fold 2 - nn auc: 0.858492
===== fold 3 =====
fold 3 - nn auc: 0.856943
===== fold 4 =====
fold 4 - nn auc: 0.856820
oof nn_auc = 0.857330068069861


In [17]:
sub[target]=nn_pred
sub.to_csv('submission.csv', index=False)

sub

,id,target
0,1000000,0.687326
1,1000001,0.271514
2,1000002,0.931397
3,1000003,0.816809
4,1000004,0.295359
...,...,...
499995,1499995,0.999999
499996,1499996,0.808890
499997,1499997,0.314416
499998,1499998,0.500142


# Log

/////// all average /////////

/// 128 quant normal kbins uniform with dropout, dstep=400, decreasing hidden units ///
2017 ver1
